In [6]:
import torch.nn as nn
import torch

layers = nn.ModuleList([
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 5)
])

In [3]:
print(layers[0])
print(layers[1])
print(layers[2])

Linear(in_features=10, out_features=20, bias=True)
ReLU()
Linear(in_features=20, out_features=5, bias=True)


In [4]:
layers.append(nn.Linear(5, 2))
print(len(layers))  # 4

4


In [5]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = nn.ModuleList([
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 5)
        ])

    def forward(self, x):
        for layer in self.layers:
            # 每个模块单独前向传播
            x = layer(x)
        return x

model = MyModel()
print(model)

MyModel(
  (layers): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=5, bias=True)
  )
)


In [7]:
model_seq = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 5)
)

output = model(torch.randn(1, 10))  # 直接传入数据即可

In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Linear(in_features, in_features),  # 第一个全连接层
            nn.ReLU(),
            nn.Linear(in_features, in_features)   # 第二个全连接层
        ])
    
    def forward(self, x):
        identity = x  # 保存输入 x（残差连接）
        
        for layer in self.layers:
            x = layer(x)  # 逐层执行计算
        # Residual connect: output = x + f(x)
        return x + identity  # 添加残差连接

# 测试
x = torch.randn(1, 5)  # 生成输入数据
res_block = ResidualBlock(5)  # 创建残差块
output = res_block(x)  # 计算输出
print(output)

tensor([[-0.4142,  0.4323,  1.8367,  0.6806, -0.0357]], grad_fn=<AddBackward0>)


In [14]:
class ResidualNetwork(nn.Module):
    def __init__(self, num_blocks, in_features):
        super().__init__()
        self.res_blocks = nn.ModuleList([ResidualBlock(in_features) for _ in range(num_blocks)])

    def forward(self, x):
        for block in self.res_blocks:
            x = block(x)  # 每个残差块都会执行跳跃连接
        return x

# 测试
x = torch.randn(1, 5)
model = ResidualNetwork(3, 5)  # 3 层残差网络
output = model(x)
print(output)

print(model.children)


tensor([[-1.4532, -0.2639, -0.7839, -1.1680, -0.8791]], grad_fn=<AddBackward0>)
<bound method Module.children of ResidualNetwork(
  (res_blocks): ModuleList(
    (0-2): 3 x ResidualBlock(
      (layers): ModuleList(
        (0): Linear(in_features=5, out_features=5, bias=True)
        (1): ReLU()
        (2): Linear(in_features=5, out_features=5, bias=True)
      )
    )
  )
)>
